# Training TinyImagenet over VGG16

In [ ]:
!pip install git+https://github.com/keras-team/keras-contrib.git

In [ ]:
import keras
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import Adam
import numpy as np
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras_contrib.callbacks import CyclicLR
import imgaug as ia
from imgaug import augmenters as iaa

In [ ]:
# Hyperparameters
batch_size = 128
num_classes = 200
epochs = 48
num_train = 100000
num_validation = 10000
img_height = 64
img_width = 64
channels = 3

# Model Building

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

input_shape = (64, 64, 3)

model = Sequential([
    Conv2D(64, (3, 3), input_shape=input_shape, padding='same',
           activation='relu'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(200, activation='softmax')
])

model.summary()

# Load Dataset & Pre-process

### Dataset: [TinyImagenet](https://tiny-imagenet.herokuapp.com/)

###  Note: Generate CCP attacked images and include in the training images for Augmented CCP training

In [ ]:
import pandas as pd

# Read Validation Dataset
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, 
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

In [ ]:
# Defining Customized Imagedatagenerator using imgaug library
def CustomImageDataGen(input_img):
  # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
  # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
  # image.
  sometimes = lambda aug: iaa.Sometimes(0.5, aug)
  
  seq = iaa.Sequential([
      iaa.Fliplr(0.5), # horizontal flips
      iaa.Flipud(0.2), # vertical flips
      
      # Small gaussian blur with random sigma between 0 and 0.5.
      # But we only blur about 50% of all images.
      sometimes(iaa.GaussianBlur(sigma=(0, 2.0))),
      
      # crop images by -10% to 20% of their height/width
      sometimes(iaa.CropAndPad(
          percent=(-0.1, 0.2),
          pad_mode=ia.ALL,
          pad_cval=(0, 255)
        )),
      
      # Apply affine transformations to some of the images
      # - scale to 80-120% of image height/width (each axis independently)
      # - translate by -20 to +20 relative to height/width (per axis)
      # - rotate by -45 to +45 degrees
      # - shear by -16 to +16 degrees
      # - order: use nearest neighbour or bilinear interpolation (fast)
      # - mode: use any available mode to fill newly created pixels
      #         see API or scikit-image for which modes are available
      # - cval: if the mode is constant, then use a random brightness
      #         for the newly created pixels (e.g. sometimes black,
      #         sometimes white)
      sometimes(iaa.Affine(
          scale={"x": (0.8, 1.5), "y": (0.8, 1.5)},
          translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
          rotate=(-45, 45),
          shear=(-16, 16),
          order=[0, 1],
          cval=(0, 255),
          mode=ia.ALL
      )),
      
      #drop 2-5% percent of the original size, leading to large dropped
      # rectangles.
      sometimes(iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    )),
                
      # Make some images brighter and some darker.
      # In 20% of all cases, we sample the multiplier once per channel,
      # which can end up changing the color of the images.
      sometimes(iaa.Multiply((0.8, 1.2), per_channel=0.2)),
      
      #Improve or worsen the contrast of images.
      #Comment it out after third model run (extreme saturation)
      sometimes(iaa.ContrastNormalization((0.75, 1.5), per_channel=0.5)), 
     ],
     # do all of the above augmentations in random order
     random_order = True) # apply augmenters in random order
  
  output_img = seq.augment_image(input_img)
  return output_img

train_datagen = ImageDataGenerator(rescale=1/255., preprocessing_function = CustomImageDataGen)
valid_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, seed=101)

In [ ]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', 
                                                         x_col='File', y_col='Class', 
                                                         target_size=(img_width, img_height),
                                                         class_mode='categorical', 
                                                         batch_size=batch_size, 
                                                         shuffle=False, seed=101)

# Model Compile and Train

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= keras.optimizers.Adam(lr= 0.0001, epsilon=1e-08),
              metrics=['accuracy'])

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa

# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpointer = ModelCheckpoint(filepath="vgg16_tinyimagenet1.h5", verbose=1, save_best_only=True, monitor="val_accuracy")

# Fit the Model
model.fit_generator(train_generator,
                    epochs=epochs,
                    steps_per_epoch= num_train // batch_size,
                    validation_steps= num_validation // batch_size,
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpointer]
                   )
model.save('vgg16_tinyimagenet1_final.h5')

In [ ]:
# load the model best model
from keras.models import load_model
new_model = load_model("vgg16_tinyimagenet1.h5")

In [ ]:
# callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpointer_2 = ModelCheckpoint(filepath="vgg16_tinyimagenet2.h5", verbose=1, save_best_only=True, monitor="val_accuracy")
# fit the model
new_model.fit_generator(train_generator,
                        epochs=12,
                        steps_per_epoch= num_train // batch_size,
                        validation_steps= num_validation // batch_size,
                        validation_data=validation_generator,
                        verbose=1, callbacks=[clr, checkpointer_2]
                       )
new_model.save('vgg16_tinyimagenet2_final.h5')

In [ ]:
# load the model best model
from keras.models import load_model
new_model = load_model("vgg16_tinyimagenet2.h5")

In [ ]:
# callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpointer_2 = ModelCheckpoint(filepath="vgg16_tinyimagenet3.h5", verbose=1, save_best_only=True, monitor="val_accuracy")
# fit the model
new_model.fit_generator(train_generator,
                        epochs=12,
                        steps_per_epoch= num_train // batch_size,
                        validation_steps= num_validation // batch_size,
                        validation_data=validation_generator,
                        verbose=1, callbacks=[clr, checkpointer_2]
                       )
new_model.save('vgg16_tinyimagenet3_final.h5')

# Evaluate Model

In [ ]:
scores = extended_model.evaluate(validation_generator)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1]*100)

# Testing on CCP Attack
### Testing the CALTECH test dataset with s=1 and b=30
#### (CCP_F and CCP_V)

In [ ]:
# Load Model
import keras
model = keras.models.load_model('vgg16_tinyimagenet3.h5')

##### Note: Generate the CCP attacked images on validation set and use them for evaluating the model.

In [ ]:
import pandas as pd

# Read Validation Dataset
val_data_transformed = pd.read_csv('./tranformed-tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, 
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

valid_datagen = ImageDataGenerator(rescale=1/255.)
validation_generator_transformed = valid_datagen.flow_from_dataframe(val_data_transformed, directory='./tranformed-tiny-imagenet-200/val/images/', 
                                                         x_col='File', y_col='Class', 
                                                         target_size=(img_width, img_height),
                                                         class_mode='categorical', 
                                                         batch_size=batch_size, 
                                                         shuffle=False, seed=101)

In [ ]:
scores = extended_model.evaluate(validation_generator_transformed)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1]*100)